# 트랜스포머로 DSA 데이터 분류

In [62]:
import pandas as pd

df= pd.read_csv(r"C:\Users\DoHyeonjik\GachonUniv\3-2\datasets\DL\DSA_features.csv")
df.shape

(9120, 272)

In [63]:
df['activity'].value_counts()

activity
sitting                    480
walkingTreadmillIncline    480
jumping                    480
rowing                     480
cyclingVertical            480
cyclingHorizontal          480
crossTrainer               480
stepper                    480
runningTreadmill           480
walkingTreadmillFlat       480
standing                   480
walkingLot                 480
movingInElevator           480
standingInElevatorStill    480
decendingStairs            480
ascendingStairs            480
lyingRigh                  480
lyingBack                  480
basketBall                 480
Name: count, dtype: int64

In [64]:
#df_RA랑 LA로 나누기
df_RA = df.filter(regex='RA_')
df_LA = df.filter(regex='LA_')
print(df_RA.shape, df_LA.shape)

(9120, 54) (9120, 54)


In [65]:
# 데이터에 레이블 삽입
df_RA.insert(54, 'activity', df['activity'])


In [66]:
# 레이블 값을 라벨인코딩
from sklearn.preprocessing import LabelEncoder
le= LabelEncoder()
df_RA['activity'] = le.fit_transform(df_RA['activity'])

C:\TempFolder\ipykernel_17992\2372902343.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_RA['activity'] = le.fit_transform(df_RA['activity'])


In [67]:
#레이블 분리 
df_RA_label = df_RA['activity']
df_RA_data = df_RA.drop('activity', axis=1)

In [68]:
#traindata와 testdata로 분리
from sklearn.model_selection import train_test_split

x_train_RA, x_test_RA, y_train_RA, y_test_RA = train_test_split(df_RA_data, df_RA_label, test_size=0.25, random_state=21)
print(x_train_RA.shape)
print(x_test_RA.shape)
print(y_train_RA.shape)

(6840, 54)
(2280, 54)
(6840,)


In [69]:
from collections import Counter
import numpy as np

def split_sequences(sequences, n_steps):
    X, y = list(), list()  # 빈 리스트를 생성하여 시퀀스 데이터와 레이블을 담을 공간을 만듦
    for i in range(len(sequences)):  # 전체 시퀀스 데이터를 순회
        # find the end of this pattern
        end_ix = i + n_steps  # 현재 인덱스(i)에서 n_steps만큼 떨어진 시퀀스의 끝을 계산
        # check if we are beyond the dataset
        if end_ix > len(sequences):  # 시퀀스 끝이 데이터의 범위를 넘어서는지 확인
            break  # 범위를 넘으면 루프 종료
        # gather input (X) and output parts (y)
        seq_x = sequences[i:end_ix, :-1]  # 입력 데이터 (특징 데이터)
        seq_y_values = sequences[i:end_ix, -1]  # 시퀀스 동안의 출력 데이터 (레이블들)
            
        # 가장 빈번하게 나온 레이블 찾기
        most_common_label = Counter(seq_y_values).most_common(1)[0][0]
        if i ==0: print(most_common_label)
        
        X.append(seq_x)  # 입력 데이터 추가
        y.append(most_common_label)  # 가장 많이 나온 레이블 추가
    
    return np.array(X), np.array(y)  # 리스트를 numpy 배열로 변환하여 반환

In [70]:
#Merge train and test x/y data to apply sequence transformation function
#훈련 데이터와 테스트 데이터의 레이블과 데이터를 병합 -> 시퀀스 변형을 위해서
y_train_array_RA = np.array(y_train_RA) #RA의 훈련 레이블을 넘파이로 바꾸고
train_set_RA = np.c_[x_train_RA, y_train_array_RA] #RA의 훈련데이터와 훈련레이블을 병합
print(train_set_RA.shape)

y_test_array_RA = np.array(y_test_RA) #RA의 테스트 레이블을 넘파이로 바꾸고
test_set_RA = np.c_[x_test_RA, y_test_array_RA] #RA의 테스트데이터와 테스트레이블을 병합

(6840, 55)


In [71]:
#split_sequence 적용
n_step = 3 

x_train_seq_RA, y_train_seq_RA = split_sequences(train_set_RA, n_step) # RA 훈련 데이터
print(x_train_seq_RA.shape, y_train_seq_RA.shape)

x_test_seq_RA, y_test_seq_RA = split_sequences(test_set_RA, n_step) # RA 훈련 레이블
print(x_test_seq_RA.shape, y_test_seq_RA.shape)

x_train_seq_LA, y_train_seq_LA = split_sequences(train_set_LA, n_step) # LA 훈련 데이터
print(x_train_seq_LA.shape, y_train_seq_LA.shape)

x_test_seq_LA, y_test_seq_LA = split_sequences(test_set_LA, n_step) # RA 훈련 레이블
print(x_test_seq_LA.shape, y_test_seq_LA.shape)


9.0
(6838, 3, 54) (6838,)
7.0
(2278, 3, 54) (2278,)
9.0
(6838, 3, 54) (6838,)
7.0
(2278, 3, 54) (2278,)


In [72]:
#레이블 원핫인코딩
from keras.utils import to_categorical

# y_train_seq_RA = to_categorical(y_train_seq_RA) #RA 훈련 레이블 
# 
# y_test_seq_RA = to_categorical(y_test_seq_RA) #RA 테스트 레이블

print(y_train_seq_RA.shape)

(6838,)


In [73]:
# 트랜스포머

from tensorflow import keras
from keras.layers import MultiHeadAttention
from keras import layers
def transformer_encoder(inputs, head_size, num_heads, ff_dim, dropout=0):
    x=layers.MultiHeadAttention(key_dim = head_size, num_heads = num_heads, dropout=dropout)(inputs, inputs)
    x=layers.Dropout(dropout)(x)
    x=layers.LayerNormalization(epsilon=1e-6)(x)
    res = x + inputs

    x = layers.Conv1D(filters = ff_dim, kernel_size=1, activation='relu')(res)
    x = layers.Dropout(dropout)(x)
    x = layers.Conv1D(filters=inputs.shape[-1], kernel_size=1)(x)
    x = layers.LayerNormalization(epsilon=1e-6)(x)
    return x + res
                     
    

In [74]:
# 모델 생성 함수
def build_model(input_shape, head_size, num_heads, ff_dim, num_transformer_blocks, mlp_units, dropout=0, mlp_dropout=0):
    inputs = keras.Input(shape = input_shape)
    x= inputs
    for _ in range(num_transformer_blocks):
        x = transformer_encoder(x, head_size, num_heads, ff_dim, dropout)

    x = layers.GlobalAveragePooling1D(data_format="channels_last")(x)
    for dim in mlp_units:
        x = layers.Dense(dim, activation='relu')(x)
        x = layers.Dropout(mlp_dropout)(x)
    outputs = layers.Dense(n_classes, activation='softmax')(x)
    return keras.Model(inputs, outputs)

In [75]:
input_shape = x_train_seq_RA.shape[1:]
n_classes = 19

model = build_model(input_shape, head_size= 256, num_heads=4, ff_dim=4, num_transformer_blocks=1, mlp_units=[128], mlp_dropout=0.4, dropout=0.25)

model.compile(loss='sparse_categorical_crossentropy', optimizer='adam', metrics=['sparse_categorical_accuracy'])

model.summary()

Model: "functional_7"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                  ┃ Output Shape              ┃         Param # ┃ Connected to               ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│ input_layer_8 (InputLayer)    │ (None, 3, 54)             │               0 │ -                          │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ multi_head_attention_4        │ (None, 3, 54)             │         224,310 │ input_layer_8[0][0],       │
│ (MultiHeadAttention)          │                           │                 │ input_layer_8[0][0]        │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ dropout_15 (Dropout)          │ (None, 3, 54)             │               0 │ multi_head_attention_4[0]… │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ layer_normalization_7         │ (None, 3, 54)             │             108 │ dropout_15[0][0]           │
│ (LayerNormalization)          │                           │                 │                            │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ add_7 (Add)                   │ (None, 3, 54)             │               0 │ layer_normalization_7[0][… │
│                               │                           │                 │ input_layer_8[0][0]        │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ conv1d_7 (Conv1D)             │ (None, 3, 4)              │             220 │ add_7[0][0]                │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ dropout_16 (Dropout)          │ (None, 3, 4)              │               0 │ conv1d_7[0][0]             │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ conv1d_8 (Conv1D)             │ (None, 3, 54)             │             270 │ dropout_16[0][0]           │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ layer_normalization_8         │ (None, 3, 54)             │             108 │ conv1d_8[0][0]             │
│ (LayerNormalization)          │                           │                 │                            │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ add_8 (Add)                   │ (None, 3, 54)             │               0 │ layer_normalization_8[0][… │
│                               │                           │                 │ add_7[0][0]                │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ global_average_pooling1d_3    │ (None, 54)                │               0 │ add_8[0][0]                │
│ (GlobalAveragePooling1D)      │                           │                 │                            │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ dense_6 (Dense)               │ (None, 128)               │           7,040 │ global_average_pooling1d_… │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ dropout_17 (Dropout)          │ (None, 128)               │               0 │ dense_6[0][0]              │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ dense_7 (Dense)               │ (None, 19)                │           2,451 │ dropout_17[0][0]           │
└───────────────────────────────┴───────────────────────────┴───────────────

 Total params: 234,507 (916.04 KB)

 Trainable params: 234,507 (916.04 KB)

 Non-trainable params: 0 (0.00 B)

In [76]:
history = model.fit(x_train_seq_RA, y_train_seq_RA, epochs=150, batch_size =64, validation_split=0.2) 

Epoch 1/150
86/86 ━━━━━━━━━━━━━━━━━━━━ 7s 31ms/step - loss: 4.5951 - sparse_categorical_accuracy: 0.0854 - val_loss: 2.6944 - val_sparse_categorical_accuracy: 0.2054
Epoch 2/150
86/86 ━━━━━━━━━━━━━━━━━━━━ 2s 25ms/step - loss: 2.8910 - sparse_categorical_accuracy: 0.1514 - val_loss: 2.5950 - val_sparse_categorical_accuracy: 0.2434
Epoch 3/150
86/86 ━━━━━━━━━━━━━━━━━━━━ 2s 27ms/step - loss: 2.7030 - sparse_categorical_accuracy: 0.1857 - val_loss: 2.4713 - val_sparse_categorical_accuracy: 0.2683
Epoch 4/150
86/86 ━━━━━━━━━━━━━━━━━━━━ 2s 26ms/step - loss: 2.5522 - sparse_categorical_accuracy: 0.2278 - val_loss: 2.4042 - val_sparse_categorical_accuracy: 0.2727
Epoch 5/150
86/86 ━━━━━━━━━━━━━━━━━━━━ 2s 27ms/step - loss: 2.5266 - sparse_categorical_accuracy: 0.2279 - val_loss: 2.3331 - val_sparse_categorical_accuracy: 0.2887
Epoch 6/150
86/86 ━━━━━━━━━━━━━━━━━━━━ 2s 27ms/step - loss: 2.4129 - sparse_categorical_accuracy: 0.2532 - val_loss: 2.2465 - val_sparse_categorical_accuracy: 0.3070
Epoc